<a href="https://colab.research.google.com/github/jiyeon712/Medicine_classification/blob/main/Medicine_MultiClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 모델 별 정확도
- Cnn with vgg 16 : 99.4%
- Resnet : 99.01%
- Cnn : 98.68%

# 데이터 불러오기

In [ ]:
import os
import glob

# 파일 경로 설정
image_files = glob.glob("/content/drive/MyDrive/MedicineContest/drug&medicine/*/*")

# 폴더명 추출
folder_names = {os.path.basename(os.path.dirname(file_path)) for file_path in image_files}

print('파일명 : ', folder_names)
print('파일 개수 :' ,len(folder_names))

파일명 :  {'28 Methylenedioxypyrovalerone,MDPV', '259 Diazepam', '282 5-MeO-DiPT', '40122 Uristat', '258 Lorazepam', '11 Locaserin', '238 Mexazolam', '40767 Gestaren Tab', '224 Bromazepam', '128 Pentazocine', '114 Flurazepam', '139 Fenetylline', '162 Carisoprodol', '9 AL-LAD', '41107 Anacox Cap', '157 Kratom', '40720 New Glia Tab', '1 Reminazolam', '169 Zopiclone', '166 Zipeprol', '156 Clonazepam', '22 Etizolam', '168 Zolpidem', '193 Ethyl Loflazepate', '59 4,4_DMAR', '40990 Sebaco Hct Tab', '30 W-18', '41207 Vtamin Tab', '170 Ibogaine', '161 Camazepam', '41172 Levetiracetam', '41170 Leviepill Tab', '303 Phenibut', '137 Pemoline', '29002 Opiqutan Soft Cap', '40953 Canagabarotin Cap', '152 Chlordiazepoxide', '107 Halazepam', '195 Ethchlorvynol', '34342 Welood Soft Cap', '295 2C-B', '245 Methylphenidate', '40792 Diacell Cap', '253 Mazindol', '40949 Afental CR Tab', '37990 Multi-Q10', '147 Temazepam', '41327 Rabeprazole Sodium', '94 5-mapb', '78 Phenazepam', '40991 Sebaco Hct Tab', '41169 Ra

In [ ]:
import cv2
import numpy as np
import os

# 이미지 파일 경로 설정
image_files = glob.glob("/content/drive/MyDrive/MedicineContest/drug&medicine/*/*")

# 이미지와 라벨을 저장할 리스트 생성
images = []  # 이미지 데이터를 저장할 리스트
labels = []  # 라벨 데이터를 저장할 리스트

# 이미지 파일을 읽어서 이미지 데이터(images)와 라벨 데이터(labels)에 저장
for file_path in image_files:
    # 이미지 파일의 경로에서 라벨(폴더 이름)을 추출
    label = os.path.basename(os.path.dirname(file_path))
    # 이미지 파일 읽기
    img = cv2.imread(file_path)
    # 이미지 크기 조정 (선택적)
    img = cv2.resize(img, (150, 150))
    # 이미지 데이터(images)와 라벨 데이터(labels)에 추가
    images.append(img)
    labels.append(label)

# 리스트를 넘파이 배열로 변환
images = np.array(images)
labels = np.array(labels)

# 데이터셋의 크기 확인
print("이미지 데이터의 크기:", images.shape)
print("라벨 데이터의 크기:", labels.shape)

이미지 데이터의 크기: (3796, 150, 150, 3)
라벨 데이터의 크기: (3796,)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# 라벨 인코더 생성
label_encoder = LabelEncoder()

# 라벨 데이터를 숫자 형태로 변환
labels_encoded = label_encoder.fit_transform(labels)

## Train/Test로 구분하기

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

x_train, x_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)
x_train.shape,y_train.shape

((3036, 150, 150, 3), (3036,))

# 모델 설계

### CNN

In [ ]:
x_train_scaled=x_train/255.0
x_test_scaled=x_test/255.0

### DNN 모델에서는 실행해야함.
# x_train_scaled = x_train_scaled.reshape(-1,150*150*3)
# x_test_scaled=x_test_scaled.reshape(-1,150*150*3)

In [ ]:
import numpy as np
import tensorflow as tf
from keras import Sequential, Input
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Input(shape = (150,150,3)))


model.add(Conv2D(32, kernel_size = 3, activation='relu',padding='same'))
model.add(MaxPooling2D(2))

model.add(Conv2D(64, kernel_size=3, activation='relu',padding='same'))
model.add(MaxPooling2D(2))

# 세 번째 컨볼루션 층 추가 : 256 필터
model.add(Conv2D(256, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D(2))

# 네 번째 컨볼루션 층: 512 필터
model.add(Conv2D(512, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling2D(2))

model.add(Flatten())

# Dense 층: 100 -> 200 노드로 증가
model.add(Dense(200, activation='relu'))
model.add(Dropout(rate=0.4))


model.add(Dense(68, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 75, 75, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 37, 37, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 37, 37, 256)       147712    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 18, 18, 256)       0

### CNN 모델 학습

In [ ]:
from tensorflow.keras import optimizers

# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# 모델 학습
history = model.fit(x_train_scaled, y_train,
                    validation_split=0.3,
                    batch_size=32,
                    epochs=100,
                    verbose=1)

Epoch 1/100
67/67 [==============================] - 13s 89ms/step - loss: 4.0665 - accuracy: 0.0480 - val_loss: 3.2736 - val_accuracy: 0.1131
Epoch 2/100
67/67 [==============================] - 3s 41ms/step - loss: 2.9593 - accuracy: 0.1991 - val_loss: 2.4451 - val_accuracy: 0.3304
Epoch 3/100
67/67 [==============================] - 3s 45ms/step - loss: 2.2358 - accuracy: 0.3713 - val_loss: 1.6742 - val_accuracy: 0.4852
Epoch 4/100
67/67 [==============================] - 3s 41ms/step - loss: 1.7248 - accuracy: 0.4955 - val_loss: 1.2533 - val_accuracy: 0.6136
Epoch 5/100
67/67 [==============================] - 3s 42ms/step - loss: 1.3419 - accuracy: 0.5962 - val_loss: 1.0730 - val_accuracy: 0.6542
Epoch 6/100
67/67 [==============================] - 3s 48ms/step - loss: 1.0960 - accuracy: 0.6612 - val_loss: 0.9796 - val_accuracy: 0.6948
Epoch 7/100
67/67 [==============================] - 3s 45ms/step - loss: 0.9225 - accuracy: 0.7285 - val_loss: 0.6356 - val_accuracy: 0.7870
Epoch

In [ ]:
# 모델 평가
model.evaluate(x_test_scaled,y_test)

24/24 [==============================] - 1s 43ms/step - loss: 0.0718 - accuracy: 0.9868


[0.07182274013757706, 0.9868420958518982]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# 모델 예측 및 평가 지표 계산
y_pred = model.predict(x_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

class_report = classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_)
print("Classification Report:\n", class_report)

24/24 [==============================] - 0s 12ms/step
Confusion Matrix:
 [[ 8  0  0 ...  0  0  0]
 [ 0  5  0 ...  0  0  0]
 [ 0  0 23 ...  0  0  0]
 ...
 [ 0  0  0 ...  7  0  0]
 [ 0  0  0 ...  0  5  0]
 [ 0  0  0 ...  0  0  4]]
Classification Report:
                                     precision    recall  f1-score   support

                     1 Reminazolam       1.00      1.00      1.00         8
                     107 Halazepam       1.00      0.71      0.83         7
                      11 Locaserin       1.00      1.00      1.00        23
                    114 Flurazepam       1.00      1.00      1.00         5
                 116 Flunitrazepam       0.94      1.00      0.97        15
                       12 Quazepam       1.00      1.00      1.00        36
                   128 Pentazocine       1.00      1.00      1.00        10
               133 Phendimetrazine       1.00      1.00      1.00        19
                      137 Pemoline       1.00      1.00      1

1. Macro Average (macro avg):

정의: 모든 클래스의 지표를 계산한 후, 클래스의 비율을 고려하지 않고 단순 평균을 구함
특징: 각 클래스의 중요도를 동일하게 취급. 클래스 불균형이 있는 데이터셋에서는 소수 클래스의 성능에 더 민감하게 반응.

2. Weighted Average (weighted avg):

정의: 각 클래스의 지표를 계산한 후, 해당 클래스의 샘플 수를 가중치로 하여 가중 평균을 구함
특징: 클래스의 불균형을 고려. 즉, 샘플 수가 많은 클래스가 전체 평균에 더 큰 영향을 미침.


### vgg16 transfer

In [ ]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Model
import tensorflow.keras

# VGG모델 불러오기
vgg_base = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(150,150,3))

vgg_base.summary()

58889256/58889256 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

### vgg 모델 설계

In [ ]:
from keras.models import Sequential
from keras import layers

# 모델 생성
model_vgg = Sequential()
model_vgg.add(vgg_base)
model_vgg.add(layers.Flatten())
model_vgg.add(layers.Dense(256, activation='relu'))
model_vgg.add(layers.Dropout(0.5))
model_vgg.add(layers.Dense(68, activation='softmax'))

In [ ]:
# VGG16의 Convolutional Layer는 이미 학습되어 있으므로 학습되지 않도록 설정
vgg_base.trainable = False

In [ ]:
# 모델 컴파일
model_vgg.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['accuracy'])

# 모델 학습
history_vgg = model_vgg.fit(x_train_scaled, y_train, batch_size=32, epochs=50, validation_split=0.3)

Epoch 1/50
67/67 [==============================] - 17s 187ms/step - loss: 3.3009 - accuracy: 0.2296 - val_loss: 2.0815 - val_accuracy: 0.4577
Epoch 2/50
67/67 [==============================] - 6s 97ms/step - loss: 1.7558 - accuracy: 0.5125 - val_loss: 1.0536 - val_accuracy: 0.7355
Epoch 3/50
67/67 [==============================] - 6s 96ms/step - loss: 1.1479 - accuracy: 0.6696 - val_loss: 0.6887 - val_accuracy: 0.7925
Epoch 4/50
67/67 [==============================] - 7s 110ms/step - loss: 0.8339 - accuracy: 0.7544 - val_loss: 0.5807 - val_accuracy: 0.8463
Epoch 5/50
67/67 [==============================] - 6s 97ms/step - loss: 0.6154 - accuracy: 0.8198 - val_loss: 0.3924 - val_accuracy: 0.8771
Epoch 6/50
67/67 [==============================] - 7s 108ms/step - loss: 0.4665 - accuracy: 0.8522 - val_loss: 0.2328 - val_accuracy: 0.9451
Epoch 7/50
67/67 [==============================] - 7s 106ms/step - loss: 0.3776 - accuracy: 0.8847 - val_loss: 0.1955 - val_accuracy: 0.9594
Epoch 8/

In [ ]:
# 모델 평가
test_loss, test_acc = model_vgg.evaluate(x_test_scaled, y_test)
print(f"Test Accuracy: {test_acc}")

# 예측값 계산
y_pred = model_vgg.predict(x_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

# 혼동 행렬
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_)
print("Classification Report:\n", class_report)

24/24 [==============================] - 4s 180ms/step - loss: 0.0171 - accuracy: 0.9947
Test Accuracy: 0.9947368502616882
24/24 [==============================] - 2s 63ms/step
Confusion Matrix:
 [[ 8  0  0 ...  0  0  0]
 [ 0  7  0 ...  0  0  0]
 [ 0  0 23 ...  0  0  0]
 ...
 [ 0  0  0 ...  7  0  0]
 [ 0  0  0 ...  0  5  0]
 [ 0  0  0 ...  0  0  4]]
Classification Report:
                                     precision    recall  f1-score   support

                     1 Reminazolam       1.00      1.00      1.00         8
                     107 Halazepam       1.00      1.00      1.00         7
                      11 Locaserin       1.00      1.00      1.00        23
                    114 Flurazepam       1.00      1.00      1.00         5
                 116 Flunitrazepam       1.00      1.00      1.00        15
                       12 Quazepam       1.00      1.00      1.00        36
                   128 Pentazocine       1.00      1.00      1.00        10
               

In [ ]:
# 모델 저장
model_vgg.save('vgg16_transfer_1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### DNN

In [ ]:
# 데이터 전처리
x_train_scaled=x_train/255.0
x_test_scaled=x_test/255.0
x_train_scaled = x_train_scaled.reshape(-1,150*150*3)
x_test_scaled=x_test_scaled.reshape(-1,150*150*3)

In [ ]:
import numpy as np
import tensorflow as tf
from keras import Sequential, Input
from keras.layers import Dense, Dropout
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# 모델 구성
model_d = Sequential()
model_d.add(Input(shape=(150*150*3, )))

# model.add(Flatten())

model_d.add(Dense(64,activation='relu'))
model_d.add(Dense(32,activation='relu'))
model_d.add(Dense(16,activation='relu'))
model_d.add(Dense(32,activation='relu'))
model_d.add(Dense(68,activation='softmax'))

In [ ]:
model_d.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics='accuracy')
history = model_d.fit(x_train_scaled, y_train, epochs=100, batch_size=32,validation_split=0.3)

Epoch 1/100
67/67 [==============================] - 4s 26ms/step - loss: 4.5361 - accuracy: 0.0466 - val_loss: 4.0439 - val_accuracy: 0.0582
Epoch 2/100
67/67 [==============================] - 1s 11ms/step - loss: 3.8915 - accuracy: 0.0814 - val_loss: 3.7775 - val_accuracy: 0.0615
Epoch 3/100
67/67 [==============================] - 1s 13ms/step - loss: 3.6129 - accuracy: 0.1144 - val_loss: 3.4879 - val_accuracy: 0.1010
Epoch 4/100
67/67 [==============================] - 1s 12ms/step - loss: 3.3658 - accuracy: 0.1289 - val_loss: 3.2618 - val_accuracy: 0.1175
Epoch 5/100
67/67 [==============================] - 1s 11ms/step - loss: 3.1422 - accuracy: 0.1699 - val_loss: 3.0310 - val_accuracy: 0.1636
Epoch 6/100
67/67 [==============================] - 1s 11ms/step - loss: 2.9560 - accuracy: 0.1896 - val_loss: 2.9083 - val_accuracy: 0.1998
Epoch 7/100
67/67 [==============================] - 1s 11ms/step - loss: 2.8107 - accuracy: 0.2259 - val_loss: 2.7546 - val_accuracy: 0.2404
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# 모델 평가
test_loss, test_acc = model_d.evaluate(x_test_scaled, y_test)
print(f"Test Accuracy: {test_acc}")

# 예측값 계산
y_pred = model_d.predict(x_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

# 혼동 행렬
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_test, y_pred_classes)
print("Classification Report:\n", class_report)

24/24 [==============================] - 0s 6ms/step - loss: 1.2788 - accuracy: 0.6434
Test Accuracy: 0.6434210538864136
24/24 [==============================] - 0s 6ms/step
Confusion Matrix:
 [[8 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 5 0]
 [0 0 0 ... 0 0 4]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.00      0.00      0.00         7
           2       0.00      0.00      0.00        23
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00        15
           5       1.00      1.00      1.00        36
           6       0.00      0.00      0.00        10
           7       0.11      0.37      0.16        19
           8       1.00      1.00      1.00        13
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00        12
          11  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### ResNet

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

resnet_base = ResNet50(include_top=False, weights='imagenet', input_shape=(150,150,3))

resnet_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 156, 156, 3)          0         ['input_5[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 75, 75, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 75, 75, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.applications import ResNet50
from keras.optimizers import RMSprop

model_resnet = Sequential()
model_resnet.add(resnet_base)
model_resnet.add(layers.Flatten())
model_resnet.add(layers.Dense(256, activation='relu'))
model_resnet.add(layers.Dropout(0.5))
model_resnet.add(layers.Dense(68, activation='softmax'))

# ResNet50의 Convolutional Layer는 이미 학습되어 있으므로 학습되지 않도록 설정
resnet_base.trainable = False

# 모델 컴파일
model_resnet.compile(loss='sparse_categorical_crossentropy', optimizer=RMSprop(learning_rate=1e-4), metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# ModelCheckpoint 콜백 생성
checkpoint_path = "best_model_resnet.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

# EarlyStopping 콜백 생성
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

# 모델 학습 시 ModelCheckpoint와 EarlyStopping 콜백 지정
history_resnet = model_resnet.fit(x_train, y_train, epochs=50, batch_size=512, validation_split=0.2, callbacks=[checkpoint, early_stopping])

Epoch 1/50
5/5 [==============================] - ETA: 0s - loss: 4.3143 - accuracy: 0.2055
Epoch 1: val_loss improved from -inf to 1.28952, saving model to best_model_resnet.h5
5/5 [==============================] - 10s 1s/step - loss: 4.3143 - accuracy: 0.2055 - val_loss: 1.2895 - val_accuracy: 0.7253
Epoch 2/50
5/5 [==============================] - ETA: 0s - loss: 1.2837 - accuracy: 0.6664
Epoch 2: val_loss did not improve from 1.28952
5/5 [==============================] - 4s 943ms/step - loss: 1.2837 - accuracy: 0.6664 - val_loss: 0.4787 - val_accuracy: 0.8717
Epoch 3/50
5/5 [==============================] - ETA: 0s - loss: 0.5987 - accuracy: 0.8340
Epoch 3: val_loss did not improve from 1.28952
5/5 [==============================] - 5s 977ms/step - loss: 0.5987 - accuracy: 0.8340 - val_loss: 0.2343 - val_accuracy: 0.9375
Epoch 4/50
5/5 [==============================] - ETA: 0s - loss: 0.3918 - accuracy: 0.9028
Epoch 4: val_loss did not improve from 1.28952
5/5 [===============

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# 모델 불러오기
#model_resnet = tf.keras.models.load_model('best_model_resnet.h5')

# 모델 평가
test_loss, test_acc = model_resnet.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc}")

# 예측값 계산
y_pred = model_resnet.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# 혼동 행렬
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_)
print("Classification Report:\n", class_report)


24/24 [==============================] - 1s 54ms/step - loss: 0.0512 - accuracy: 0.9842
Test Accuracy: 0.9842105507850647
24/24 [==============================] - 2s 50ms/step
Confusion Matrix:
 [[ 8  0  0 ...  0  0  0]
 [ 0  7  0 ...  0  0  0]
 [ 0  0 23 ...  0  0  0]
 ...
 [ 0  0  0 ...  7  0  0]
 [ 0  0  0 ...  0  5  0]
 [ 0  0  0 ...  0  0  4]]
Classification Report:
                                     precision    recall  f1-score   support

                     1 Reminazolam       1.00      1.00      1.00         8
                     107 Halazepam       1.00      1.00      1.00         7
                      11 Locaserin       1.00      1.00      1.00        23
                    114 Flurazepam       1.00      1.00      1.00         5
                 116 Flunitrazepam       1.00      1.00      1.00        15
                       12 Quazepam       1.00      1.00      1.00        36
                   128 Pentazocine       0.77      1.00      0.87        10
               1